In [821]:
import sql_tools
import pandas as pd

In [822]:
cnx, cursor = sql_tools.create_session_cursor()
result = sql_tools.execute_fetchall(cursor, 'SELECT * from food ', tuple())
result = pd.DataFrame(result)
result.head()


,0,1,2,3,4,5,6
0,1,1,荤菜,辣椒炒肉,42.0,1,82
1,2,1,荤菜,小炒黄牛肉,68.0,1,87
2,3,1,荤菜,剁椒鱼头,96.0,1,92
3,4,1,荤菜,口味鸡,88.0,1,54
4,5,1,荤菜,烟笋炒腊肉,48.0,1,44


In [823]:
Order = sql_tools.execute_fetchall(cursor, 'SELECT * from `order`;', tuple())
Order = pd.DataFrame(Order)
Order.head()

,0,1,2,3,4
0,1,1,1,2023-03-04 11:49:05,9
1,2,1,2,2023-03-15 18:31:36,4
2,3,1,3,2023-03-06 18:00:21,6
3,4,1,4,2023-03-09 11:09:10,10
4,5,1,5,2023-03-02 19:00:30,3


In [824]:
#支持度=购买了A和B商品（集合G）的人数/所有购买过商品（集合U）的人数
#Formula:   Support = The number of order inlude dish A and B / The number of orders in the restaurant

#Restaurant 1
Order_detail = sql_tools.execute_fetchall(cursor, 'SELECT * from `order` where RESTAURANT_ID=1', tuple())
Order_detail = pd.DataFrame(Order_detail)

Order_num=len(Order_detail.index)
Order_num


600

In [825]:
import numpy as np
Dish_Relation= np.zeros((10, 10))
Order_Count = np.zeros(10)
for i in range(1,Order_num+1):
    Temp = sql_tools.execute_fetchall(cursor, 'SELECT * from order_details where ORDER_ID=%s', tuple([i]))
    Temp = pd.DataFrame(Temp)
    for j in Temp[1].tolist():
        Order_Count[j-1] += 1
        for k in Temp[1].tolist():
            Dish_Relation[j-1][k-1]+=1


In [826]:
Support_Matrix = Dish_Relation / Order_num
Support_Matrix

array([[0.80833333, 0.47333333, 0.37      , 0.25833333, 0.48666667,
        0.63166667, 0.505     , 0.28833333, 0.47833333, 0.685     ],
       [0.47333333, 0.58833333, 0.26333333, 0.2       , 0.36      ,
        0.46      , 0.35666667, 0.22666667, 0.35      , 0.49333333],
       [0.37      , 0.26333333, 0.46      , 0.145     , 0.265     ,
        0.365     , 0.27333333, 0.16333333, 0.275     , 0.38333333],
       [0.25833333, 0.2       , 0.145     , 0.31333333, 0.17833333,
        0.24833333, 0.185     , 0.11833333, 0.185     , 0.25166667],
       [0.48666667, 0.36      , 0.265     , 0.17833333, 0.595     ,
        0.48      , 0.38      , 0.21333333, 0.34833333, 0.5       ],
       [0.63166667, 0.46      , 0.365     , 0.24833333, 0.48      ,
        0.79333333, 0.485     , 0.27666667, 0.46833333, 0.67      ],
       [0.505     , 0.35666667, 0.27333333, 0.185     , 0.38      ,
        0.485     , 0.62166667, 0.22      , 0.38333333, 0.52166667],
       [0.28833333, 0.22666667, 0.1633333

In [827]:
#满足比如支持度>0.6，置信度>0.6的这些商品组合才可以认为是有关联的，值得推荐的
#支持度>0.6:
#Dish 1 and Dish 6
#Dish 1 and Dish 10
#Dish 6 and Dish 10

In [828]:
#置信度=购买了A和B商品（集合G）的人数/购买了A商品（集合A）的人数
#Confidence of A = The number of order inlude dish A and B / The number of order include dish A

Confidence_A =  Dish_Relation[0][5] / Order_Count[0]
Confidence_B =  Dish_Relation[0][5] / Order_Count[5]

#Confidence_A = 0.7814432989690722
#Confidence_B = 0.7962184873949579
#Dish 1 and Dish 6 is recommended

In [829]:
Confidence_A =  Dish_Relation[0][9] / Order_Count[0]
Confidence_B =  Dish_Relation[0][9] / Order_Count[9]

#Confidence_A = 0.8474226804123711
#Confidence_B = 0.8170974155069582
#Dish 1 and Dish 10 is recommended

In [830]:
Confidence_A =  Dish_Relation[5][9] / Order_Count[5]
Confidence_B =  Dish_Relation[5][9] / Order_Count[9]

#Confidence_A = 0.8445378151260504
#Confidence_B = 0.7992047713717694
#Dish 6 and Dish 10 is recommended

In [831]:
#Restaurant 2
Order_detail = sql_tools.execute_fetchall(cursor, 'SELECT * from `order` where RESTAURANT_ID=2', tuple())
Order_detail = pd.DataFrame(Order_detail)

Order_num=len(Order_detail.index)
Order_num

600

In [832]:
import numpy as np
Dish_Relation= np.zeros((10, 10))
Order_Count = np.zeros(10)
for i in range(601,1201):
    Temp = sql_tools.execute_fetchall(cursor, 'SELECT * from order_details where ORDER_ID=%s', tuple([i]))
    Temp = pd.DataFrame(Temp)
    for j in Temp[1].tolist():
        Order_Count[(j-1)%10] += 1
        for k in Temp[1].tolist():
            Dish_Relation[(j-1)%10][(k-1)%10]+=1

    

In [833]:
Support_Matrix = Dish_Relation / Order_num
Support_Matrix

array([[0.51166667, 0.34833333, 0.30333333, 0.40333333, 0.20666667,
        0.215     , 0.36      , 0.45      , 0.28166667, 0.35      ],
       [0.34833333, 0.69666667, 0.415     , 0.54333333, 0.26833333,
        0.26166667, 0.51      , 0.62666667, 0.37666667, 0.47333333],
       [0.30333333, 0.415     , 0.605     , 0.48666667, 0.23166667,
        0.22666667, 0.41833333, 0.54833333, 0.32666667, 0.4       ],
       [0.40333333, 0.54333333, 0.48666667, 0.785     , 0.29833333,
        0.31333333, 0.53833333, 0.71166667, 0.41      , 0.54333333],
       [0.20666667, 0.26833333, 0.23166667, 0.29833333, 0.38666667,
        0.15666667, 0.27      , 0.34166667, 0.21      , 0.25333333],
       [0.215     , 0.26166667, 0.22666667, 0.31333333, 0.15666667,
        0.40166667, 0.29666667, 0.35833333, 0.225     , 0.27833333],
       [0.36      , 0.51      , 0.41833333, 0.53833333, 0.27      ,
        0.29666667, 0.695     , 0.63333333, 0.38333333, 0.48166667],
       [0.45      , 0.62666667, 0.5483333

In [834]:
#满足比如支持度>0.6，置信度>0.6的这些商品组合才可以认为是有关联的，值得推荐的
#支持度>0.6:
#Dish 2 and Dish 8
#Dish 4 and Dish 8
#Dish 7 and Dish 8
#Dish 8 and Dish 10

In [835]:
Confidence_A =  Dish_Relation[1][7] / Order_Count[1]
Confidence_B =  Dish_Relation[1][7] / Order_Count[7]


#Confidence_A = 0.8995215311004785
#Confidence_B = 0.6937269372693727
#Dish 2 and Dish 8 is recommended

In [836]:
Confidence_A =  Dish_Relation[3][7] / Order_Count[3]
Confidence_B =  Dish_Relation[3][7] / Order_Count[7]

#Confidence_A = 0.9065817409766455
#Confidence_B = 0.7878228782287823
#Dish 4 and Dish 8 is recommended

In [837]:
Confidence_A =  Dish_Relation[6][7] / Order_Count[6]
Confidence_B =  Dish_Relation[6][7] / Order_Count[7]

#Confidence_A = 0.9112709832134293
#Confidence_B = 0.7011070110701108
#Dish 7 and Dish 8 is recommended

In [838]:
Confidence_A =  Dish_Relation[7][9] / Order_Count[7]
Confidence_B =  Dish_Relation[7][9] / Order_Count[9]

#Confidence_A = 0.6845018450184502
#Confidence_B = 0.9115479115479116
#Dish 8 and Dish 10 is recommended

In [839]:
#Restaurant 3
Order_detail = sql_tools.execute_fetchall(cursor, 'SELECT * from `order` where RESTAURANT_ID=3', tuple())
Order_detail = pd.DataFrame(Order_detail)

Order_num=len(Order_detail.index)
Order_num

600

In [840]:
import numpy as np
Dish_Relation= np.zeros((10, 10))
Order_Count = np.zeros(10)
for i in range(1201,1801):
    Temp = sql_tools.execute_fetchall(cursor, 'SELECT * from order_details where ORDER_ID=%s', tuple([i]))
    Temp = pd.DataFrame(Temp)
    for j in Temp[1].tolist():
        Order_Count[(j-1)%10] += 1
        for k in Temp[1].tolist():
            Dish_Relation[(j-1)%10][(k-1)%10]+=1

In [841]:
Support_Matrix = Dish_Relation / Order_num
Support_Matrix

array([[0.81833333, 0.555     , 0.52666667, 0.42      , 0.32333333,
        0.50833333, 0.405     , 0.64833333, 0.63666667, 0.48833333],
       [0.555     , 0.66      , 0.43833333, 0.34833333, 0.26166667,
        0.39666667, 0.32166667, 0.525     , 0.525     , 0.4       ],
       [0.52666667, 0.43833333, 0.63666667, 0.33166667, 0.235     ,
        0.40166667, 0.31666667, 0.49833333, 0.505     , 0.36666667],
       [0.42      , 0.34833333, 0.33166667, 0.51666667, 0.18333333,
        0.305     , 0.265     , 0.39833333, 0.38833333, 0.31333333],
       [0.32333333, 0.26166667, 0.235     , 0.18333333, 0.39      ,
        0.24166667, 0.19833333, 0.30666667, 0.31333333, 0.225     ],
       [0.50833333, 0.39666667, 0.40166667, 0.305     , 0.24166667,
        0.62      , 0.285     , 0.50166667, 0.47      , 0.35      ],
       [0.405     , 0.32166667, 0.31666667, 0.265     , 0.19833333,
        0.285     , 0.49833333, 0.39166667, 0.39333333, 0.30166667],
       [0.64833333, 0.525     , 0.4983333

In [842]:
#满足比如支持度>0.6，置信度>0.6的这些商品组合才可以认为是有关联的，值得推荐的
#支持度>0.6:
#Dish 1 and Dish 8
#Dish 1 and Dish 9
#Dish 8 and Dish 9

In [843]:
Confidence_A =  Dish_Relation[0][7] / Order_Count[0]
Confidence_B =  Dish_Relation[0][7] / Order_Count[7]

#Confidence_A = 0.7922606924643585
#Confidence_B = 0.8121085594989561
#Dish 1 and Dish 8 is recommended

In [844]:
Confidence_A =  Dish_Relation[0][8] / Order_Count[0]
Confidence_B =  Dish_Relation[0][8] / Order_Count[8]

#Confidence_A = 0.7780040733197556
#Confidence_B = 0.8179871520342612
#Dish 1 and Dish 9 is recommended

In [845]:
Confidence_A =  Dish_Relation[7][8] / Order_Count[7]
Confidence_B =  Dish_Relation[7][8] / Order_Count[8]

#Confidence_A = 0.7661795407098121
#Confidence_B = 0.7858672376873662
#Dish 8 and Dish 9 is recommended

In [846]:
#Restaurant 4
Order_detail = sql_tools.execute_fetchall(cursor, 'SELECT * from `order` where RESTAURANT_ID=4', tuple())
Order_detail = pd.DataFrame(Order_detail)

Order_num=len(Order_detail.index)
Order_num

600

In [847]:
import numpy as np
Dish_Relation= np.zeros((10, 10))
Order_Count = np.zeros(10)
for i in range(1801,2401):
    Temp = sql_tools.execute_fetchall(cursor, 'SELECT * from order_details where ORDER_ID=%s', tuple([i]))
    Temp = pd.DataFrame(Temp)
    for j in Temp[1].tolist():
        Order_Count[(j-1)%10] += 1
        for k in Temp[1].tolist():
            Dish_Relation[(j-1)%10][(k-1)%10]+=1

In [848]:
Support_Matrix = Dish_Relation / Order_num
Support_Matrix

array([[0.6       , 0.46833333, 0.38333333, 0.44      , 0.27666667,
        0.33      , 0.405     , 0.35666667, 0.46      , 0.30333333],
       [0.46833333, 0.78333333, 0.475     , 0.57333333, 0.375     ,
        0.39833333, 0.54      , 0.505     , 0.61666667, 0.40666667],
       [0.38333333, 0.475     , 0.62666667, 0.44833333, 0.29833333,
        0.315     , 0.43666667, 0.37833333, 0.47833333, 0.315     ],
       [0.44      , 0.57333333, 0.44833333, 0.72      , 0.325     ,
        0.36666667, 0.49666667, 0.44833333, 0.55166667, 0.35833333],
       [0.27666667, 0.375     , 0.29833333, 0.325     , 0.48166667,
        0.245     , 0.32333333, 0.29833333, 0.37666667, 0.26166667],
       [0.33      , 0.39833333, 0.315     , 0.36666667, 0.245     ,
        0.49833333, 0.35166667, 0.33      , 0.40166667, 0.24166667],
       [0.405     , 0.54      , 0.43666667, 0.49666667, 0.32333333,
        0.35166667, 0.695     , 0.455     , 0.54666667, 0.36333333],
       [0.35666667, 0.505     , 0.3783333

In [849]:
#满足比如支持度>0.6，置信度>0.6的这些商品组合才可以认为是有关联的，值得推荐的
#支持度>0.6:
#Dish 2 and Dish 9

In [850]:
Confidence_A =  Dish_Relation[1][8] / Order_Count[1]
Confidence_B =  Dish_Relation[1][8] / Order_Count[8]

#Confidence_A = 0.7872340425531915
#Confidence_B = 0.7939914163090128
#Dish 2 and Dish 9 is recommended

In [851]:
#Restaurant 5
Order_detail = sql_tools.execute_fetchall(cursor, 'SELECT * from `order` where RESTAURANT_ID=5', tuple())
Order_detail = pd.DataFrame(Order_detail)

Order_num=len(Order_detail.index)
Order_num

600

In [852]:
import numpy as np
Dish_Relation= np.zeros((10, 10))
Order_Count = np.zeros(10)
for i in range(2401,3001):
    Temp = sql_tools.execute_fetchall(cursor, 'SELECT * from order_details where ORDER_ID=%s', tuple([i]))
    Temp = pd.DataFrame(Temp)
    for j in Temp[1].tolist():
        Order_Count[(j-1)%10] += 1
        for k in Temp[1].tolist():
            Dish_Relation[(j-1)%10][(k-1)%10]+=1

In [853]:
Support_Matrix = Dish_Relation / Order_num
Support_Matrix

array([[0.77166667, 0.455     , 0.555     , 0.44833333, 0.465     ,
        0.49833333, 0.62      , 0.48833333, 0.27      , 0.45666667],
       [0.455     , 0.61      , 0.425     , 0.36      , 0.35833333,
        0.375     , 0.48666667, 0.38833333, 0.21833333, 0.36      ],
       [0.555     , 0.425     , 0.72166667, 0.425     , 0.43666667,
        0.45      , 0.57833333, 0.45333333, 0.25666667, 0.40833333],
       [0.44833333, 0.36      , 0.425     , 0.585     , 0.345     ,
        0.34833333, 0.45833333, 0.37      , 0.215     , 0.33333333],
       [0.465     , 0.35833333, 0.43666667, 0.345     , 0.59833333,
        0.35833333, 0.47833333, 0.36666667, 0.21833333, 0.35      ],
       [0.49833333, 0.375     , 0.45      , 0.34833333, 0.35833333,
        0.61833333, 0.50166667, 0.38833333, 0.24333333, 0.37833333],
       [0.62      , 0.48666667, 0.57833333, 0.45833333, 0.47833333,
        0.50166667, 0.80166667, 0.49666667, 0.28333333, 0.45666667],
       [0.48833333, 0.38833333, 0.4533333

In [854]:
#满足比如支持度>0.6，置信度>0.6的这些商品组合才可以认为是有关联的，值得推荐的
#支持度>0.6:
#Dish 1 and Dish 7

In [855]:
Confidence_A =  Dish_Relation[0][6] / Order_Count[0]
Confidence_B =  Dish_Relation[0][6] / Order_Count[6]

#Confidence_A = 0.8034557235421166
#Confidence_B = 0.7733887733887734
#Dish 1 and Dish 7 is recommended